$$HDC\ voice\ Baseline\ : ISOLET\ (voiceHD-style)$$

# Setup

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np, time, os

def set_seed(seed=123):
    import random
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(123)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cpu')

In [ ]:
# We quantize per feature (train-only min/max), then encode with iM ⊙ CiM and bundle.



# Data

## Fetch ISOLET

In [2]:
# We fetch ISOLET from OpenML.
iso = fetch_openml('isolet', version=1, as_frame=False)
X = iso['data'].astype(np.float32)  # (N, 617)
y = iso['target']

# Map labels 'A'..'Z' to 0..25
classes = sorted(np.unique(y).tolist())
label_to_id = {c:i for i,c in enumerate(classes)}
y_int = np.array([label_to_id[s] for s in y], dtype=np.int64)

# Simple stratified split: 80% train, 20% test
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y_int, test_size=0.20, random_state=123, stratify=y_int
)

N, F = X.shape
C = len(classes)
N, F, C


(7797, 617, 26)

## Quantization

In [3]:
LEVELS = 21  # We use 21 levels (0..20) as a classic VoiceHD choice.

Xmin = X_tr.min(axis=0)
Xmax = X_tr.max(axis=0)
rng  = np.maximum(Xmax - Xmin, 1e-8)

def quantize_to_levels(Xf: np.ndarray, levels: int = LEVELS) -> np.ndarray:
    Z = (Xf - Xmin) / rng
    L = np.clip(np.round(Z * (levels - 1)), 0, levels - 1).astype(np.int64)
    return L

L_tr = quantize_to_levels(X_tr, LEVELS)
L_te = quantize_to_levels(X_te, LEVELS)

L_tr_t, y_tr_t = torch.from_numpy(L_tr), torch.from_numpy(y_tr)
L_te_t, y_te_t = torch.from_numpy(L_te), torch.from_numpy(y_te)

BATCH_SIZE = 128
train_loader = DataLoader(TensorDataset(L_tr_t, y_tr_t), batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=False)
test_loader  = DataLoader(TensorDataset(L_te_t, y_te_t), batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=False)


# Utils

In [4]:
@torch.no_grad()
def bipolar_sign(x: torch.Tensor) -> torch.Tensor:
    return torch.where(x >= 0, torch.ones_like(x), -torch.ones_like(x))

def _rand_bip(shape, device=None):
    device = device or DEVICE
    r = torch.randint(0, 2, shape, device=device, dtype=torch.int8)
    return r.float().mul_(2).sub_(1)

def make_item_memory(F: int, D: int, device=None) -> torch.Tensor:
    # I assign each feature a random bipolar HV (iM).
    return _rand_bip((F, D), device=device or DEVICE)

def make_cim(levels: int, D: int, device=None) -> torch.Tensor:
    # I build CiM with progressive bit flips so adjacent levels are similar.
    device = device or DEVICE
    base = _rand_bip((D,), device=device)
    perm = torch.randperm(D, device=device)
    out  = torch.empty((levels, D), device=device, dtype=torch.float32)
    for l in range(levels):
        k  = (l * D) // max(1, levels - 1)
        hv = base.clone()
        if k > 0:
            hv[perm[:k]] = -hv[perm[:k]]
        out[l] = hv
    return out

@torch.no_grad()
def encode_isolet_batch(L_batch: torch.Tensor,  # [B, F] ints in [0..L-1]
                        iM: torch.Tensor,       # [F, D]
                        CiM: torch.Tensor,      # [L, D]
                        block_features: int = 64) -> torch.Tensor:
    # I encode each sample by binding iM[f] with CiM[level[f]] and bundling over features.
    B, F = L_batch.shape
    D = iM.shape[1]
    hv_sum = torch.zeros((B, D), device=iM.device, dtype=torch.float32)
    for f0 in range(0, F, block_features):
        f1 = min(f0 + block_features, F)
        CiM_sel = CiM[L_batch[:, f0:f1].long().to(iM.device)]  # [B, block, D]
        iM_blk  = iM[f0:f1].unsqueeze(0)                       # [1, block, D]
        hv_sum.add_((CiM_sel * iM_blk).sum(dim=1))
    return bipolar_sign(hv_sum)

@torch.no_grad()
def build_class_prototypes(loader: DataLoader,
                           iM: torch.Tensor,
                           CiM: torch.Tensor,
                           n_classes: int,
                           block_features: int = 64) -> torch.Tensor:
    D = iM.shape[1]
    accum = torch.zeros((n_classes, D), device=iM.device, dtype=torch.float32)
    for Lb, yb in loader:
        Lb = Lb.to(iM.device, non_blocking=True)
        yb = yb.to(iM.device, non_blocking=True)
        hvs = encode_isolet_batch(Lb, iM, CiM, block_features=block_features)
        for c in range(n_classes):
            m = (yb == c)
            if m.any():
                accum[c] += hvs[m].sum(dim=0)
    return bipolar_sign(accum)

@torch.no_grad()
def cosine_sim(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    a32 = a.float(); b32 = b.float()
    an = torch.linalg.norm(a32, dim=1, keepdim=True).clamp_min_(1e-8)
    bn = torch.linalg.norm(b32, dim=1, keepdim=True).clamp_min_(1e-8).T
    return (a32 @ b32.T) / (an * bn)

@torch.no_grad()
def predict_with_prototypes(L_batch: torch.Tensor,
                            iM: torch.Tensor,
                            CiM: torch.Tensor,
                            prototypes: torch.Tensor,
                            block_features: int = 64) -> torch.Tensor:
    hvs  = encode_isolet_batch(L_batch, iM, CiM, block_features=block_features)
    sims = cosine_sim(hvs, prototypes)
    return sims.argmax(dim=1)

@torch.no_grad()
def evaluate_loader(loader: DataLoader,
                    iM: torch.Tensor,
                    CiM: torch.Tensor,
                    prototypes: torch.Tensor,
                    block_features: int = 64) -> float:
    correct = total = 0
    for Lb, yb in loader:
        Lb = Lb.to(iM.device, non_blocking=True)
        yb = yb.to(iM.device, non_blocking=True)
        preds = predict_with_prototypes(Lb, iM, CiM, prototypes, block_features=block_features)
        correct += (preds == yb).sum().item()
        total   += yb.numel()
    return 100.0 * correct / total


In [ ]:
DIM = 10_000  # We typically use 8k–20k; 10k is a solid default.
iM  = make_item_memory(F, DIM, device=DEVICE)
CiM = make_cim(LEVELS, DIM, device=DEVICE)

t0 = time.time()
prototypes = build_class_prototypes(train_loader, iM, CiM, n_classes=C, block_features=64)
print("Prototypes:", prototypes.shape, "built in %.2fs" % (time.time() - t0))

acc = evaluate_loader(test_loader, iM, CiM, prototypes, block_features=64)
print(f"Test accuracy (ISOLET; VoiceHD-style): {acc:.2f}%")


# Model

## Initialize HDC space

In [5]:
DIM = 10_000  # We typically use 8k–20k; 10k is a solid default.
iM  = make_item_memory(F, DIM, device=DEVICE)
CiM = make_cim(LEVELS, DIM, device=DEVICE)

## Train prototypes

In [6]:
t0 = time.time()
prototypes = build_class_prototypes(train_loader, iM, CiM, n_classes=C, block_features=64)
print("Prototypes:", prototypes.shape, "built in %.2fs" % (time.time() - t0))


Prototypes: torch.Size([26, 10000]) built in 250.16s


In [8]:
# torch.save(prototypes, "prototypes_ISOLET.pt")

## Evaluate

In [ ]:
# torch.load("prototypes_ISOLET.pt")

In [7]:
acc = evaluate_loader(test_loader, iM, CiM, prototypes, block_features=64)
print(f"Test accuracy (ISOLET; VoiceHD-style): {acc:.2f}%")

Test accuracy (ISOLET; VoiceHD-style): 87.95%
